In [1]:
import carla, time, pygame, math, random, cv2
import numpy as np
from Common import process_image

pygame 2.6.1 (SDL 2.28.4, Python 3.7.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [10]:
import cv2
import numpy as np
import math

def draw_line_with_angle(image):
    """
    Trova il punto bianco più basso appartenente a una linea nell'immagine e traccia una linea
    per tutta l'immagine con un angolo di 70 gradi rispetto alla linea a cui appartiene il punto.

    Args:
        image (numpy.ndarray): Immagine binaria (0 e 255).

    Returns:
        numpy.ndarray: Immagine con la linea tracciata.
    """
    image_with_line = image.copy()
    height, width = image.shape

    # Trova i contorni
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return image_with_line  # Nessun contorno trovato

    # Trova il punto bianco più basso tra tutti i contorni
    lowest_point = None
    for contour in contours:
        for point in contour:
            x, y = point[0]
            if lowest_point is None or y > lowest_point[1]:
                lowest_point = (x, y)

    if lowest_point is None:
        return image_with_line  # Nessun punto bianco trovato

    # Calcola la direzione della linea con un angolo di 70 gradi
    angle_rad = math.radians(70)  # Converti 70 gradi in radianti
    dx = math.cos(angle_rad)  # Delta x (variazione lungo l'asse x)
    dy = math.sin(angle_rad)  # Delta y (variazione lungo l'asse y)

    # Calcola i punti di inizio e fine della linea
    start_point = (int(lowest_point[0] - dx * height), int(lowest_point[1] - dy * height))
    end_point = (int(lowest_point[0] + dx * height), int(lowest_point[1] + dy * height))

    # Assicurati che i punti siano entro i limiti dell'immagine
    start_point = (max(0, min(width - 1, start_point[0])), max(0, min(height - 1, start_point[1])))
    end_point = (max(0, min(width - 1, end_point[0])), max(0, min(height - 1, end_point[1])))

    # Disegna la linea sull'immagine
    cv2.line(image_with_line, start_point, end_point, (255, 255, 255), thickness=5)

    return image_with_line

# Esempio di utilizzo
# image = cv2.imread('path_to_image', cv2.IMREAD_GRAYSCALE)
# binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)[1]
# result_image = draw_line_with_angle(binary_image)
# cv2.imshow('Result', result_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()



In [11]:
def preprocess_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)
    kernel = np.ones((5, 5), np.uint8)
    mask_cleaned = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
    return mask_cleaned

In [12]:
imageURL = "output/246072.png"
image = cv2.imread(imageURL)
image = preprocess_image(image)
image = draw_line_with_angle(image)
cv2.imshow("Image with Horizontal Lines", image)
cv2.waitKey(0)
cv2.destroyAllWindows()